In [ ]:
!pip install langchain_groq langchain_core langchain_community

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is lord Ram?")
print(result.content)

Lord Rama is a major deity in Hinduism and is considered one of the most important figures in Hindu mythology. He is the seventh avatar (incarnation) of the god Vishnu and is revered as a symbol of righteousness, duty, and virtue.

According to the Hindu epic, the Ramayana, Lord Rama was a king of Ayodhya, a city in ancient India. He was born to King Dasharatha and Queen Kausalya, and was the eldest of four brothers, including Lakshmana, Bharata, and Shatrughna.

The story of Lord Rama is as follows: Rama's stepmother, Queen Kaikeyi, who was the second wife of King Dasharatha, had a son named Bharata. She wanted her son to become the king of Ayodhya, and so she manipulated King Dasharatha into banishing Rama to the forest for 14 years.

Rama, along with his wife Sita and brother Lakshmana, went into exile in the forest, where they faced many challenges and dangers. During their time in the forest, Sita was abducted by the demon king Ravana, who took her to his kingdom in Lanka.

Rama a

In [ ]:
!pip install pypdf

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 10.7 MB/s eta 

In [ ]:
!pip install sentence_transformers

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader # Keep DirectoryLoader import in case you need it elsewhere
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_groq import ChatGroq # Explicitly import ChatGroq

def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
  )
  return llm

def create_vector_db():
  # Use PyPDFLoader for a single file instead of DirectoryLoader
  loader = PyPDFLoader("/content/mental_health_Document.pdf")
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain

def main():
  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    response = qa_chain.run(query)
    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()





Intializing Chatbot.........

Human: hello
Chatbot: Hello. It's nice to meet you. I see you've been reading about mental health, specifically about how it's defined and the importance of considering an individual's social and physical environment. It's a fascinating topic, and I'm here to listen and help if you have any questions or concerns.

The text you provided highlights the complexity of mental health and how it can be influenced by various factors, including relationships, self-respect, and mastery. It also emphasizes the importance of considering the broader social and environmental context in which individuals live.

I'm struck by the quote that defines mental health as the "capacity of individuals and groups to interact with one another and the environment in ways that promote subjective wellbeing..." This definition acknowledges that mental health is not just an individual issue, but also a collective one that involves our relationships with others and the world around us.



In [ ]:
!pip install gradio


In [49]:
query = "I feel anxious all the time. What should I do?"
response = qa_chain.run(query)
print(f"Chatbot: {response}")

Chatbot: I'm so glad you reached out and shared your feelings with me. It takes a lot of courage to acknowledge and talk about anxiety. First, please know that you're not alone in feeling this way. Anxiety is a common and normal human response to perceived danger or stress, and it can be especially prevalent during certain stages of life, such as childhood and adolescence.

It's great that you're recognizing your anxiety and wanting to do something about it. The first step is to acknowledge that your feelings are valid and that it's okay to feel anxious. It doesn't mean that you're weak or flawed; it just means that you're experiencing a natural response to stress or uncertainty.

To manage your anxiety, I would recommend taking small steps towards self-care and stress reduction. This can include activities like deep breathing exercises, meditation, or yoga. Engaging in regular physical activity, such as walking or jogging, can also help reduce anxiety by releasing endorphins, also kno

In [50]:
query = input("Enter your mental health question: ")
response = qa_chain.run(query)
print(f"Chatbot: {response}")

Enter your mental health question: i am so depressed
Chatbot: I'm so sorry to hear that you're feeling depressed. It takes a lot of courage to acknowledge and share your emotions, and I'm here to listen and support you. 

First, I want to acknowledge that feeling depressed can be a really tough and isolating experience. It's like a heavy weight is pressing down on you, making it hard to find the energy or motivation to do things you normally enjoy. 

Before we go any further, I want to make sure you know that I'm not here to diagnose you or tell you what you're experiencing. But I do want to help you explore your feelings and see if there are any ways we can work together to make things feel more manageable.

Can you tell me a bit more about what's been going on and how you've been feeling lately? Is there something specific that's been weighing on your mind, or has it been a general feeling of sadness or hopelessness? 

Also, I want to remind you that it's okay to feel overwhelmed, an

In [62]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import gradio as gr
from langchain_groq import ChatGroq # Ensure ChatGroq is imported if not already


def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
  )
  return llm

def create_vector_db():
  # Use PyPDFLoader for a single file instead of DirectoryLoader
  loader = PyPDFLoader("/content/mental_health_Document.pdf")
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain

import gradio as gr

def respond_to_query(message, history):
    response = qa_chain.run(message)
    return response

chatbot = gr.ChatInterface(
    fn=respond_to_query,
    title="🧠 Mental Health Chatbot",
    description="Your supportive companion for mental wellness. Type your thoughts or questions below.",
    theme="soft",  # Optional: 'soft', 'default', 'compact' — customize as needed
    chatbot=gr.Chatbot(height=400, show_copy_button=True),
    examples=[
        "I'm feeling really anxious lately.",
        "How do I handle constant stress?",
        "What should I do if I can't sleep well?",
    ],
    submit_btn="Send",
    # Remove the unsupported parameters
    # retry_btn="Retry",
    # clear_btn="Clear",
)

chatbot.launch()



<ipython-input-62-26a27bb81676>:61: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=400, show_copy_button=True),
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:322: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3a28fec84a5c670f4c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
